# 이미지 포맷 일괄변경 프로그램 만들기

* 난이도 : ★★★☆☆☆☆☆☆☆
* 필요라이브러리: pillow, os, argparse


* 실행인자로 대상 폴더 값을 주면 해당 폴더안의 모든 이미지의 포맷을 변경하는 프로그램 입니다.
* 변경된 포맷은 convert 란 폴더를 만들어 그 안에 저장하도록 합니다.
* pillow 라이브러리를 사용합니다.
* 이전 강좌에서 사용했던 폴더 탐색 재귀 함수를 사용합니다.
* 실행인자를 처리하기 위해 argparse 라이브러리를 사용합니다.

In [ ]:
from PIL import Image
import os
import argparse


def search_dir(dirname):
    '''대상 폴더 내의 파일과 서브 폴더를 탐색하는 재귀 함수

    Args:
        dirname (str): 탐색 대상 폴더 경로

    Returns:
        list : 탐색한 결과 파일 리스트를 리턴
    '''
    # 결과를 리턴할 리스트 변수
    result_file_lists = []

    # dirname 의 경로의 파일과 폴더 목록을 구함
    filenames = os.listdir(dirname)

    # 파일과 폴더 목록 반복문
    for filename in filenames:
        # filename 에는 os.listdir(dirname) 에서 dirname 이후의 경로값과 파일명만 존재하기 때문에
        # 전체 풀 경로를 얻기 위해선 dirname 과 filename 을 합쳐야 합니다.
        full_filepath = os.path.join(dirname, filename)

        # full_filepath 가 디렉토리라면..
        if os.path.isdir(full_filepath):
            # search_dir 인 자기 자신 함수를 다시 호출합니다.(재귀함수)
            result_file_lists.extend(search_dir(full_filepath))
        # 파일인 경우 result_file_lists 에 추가
        else:
            result_file_lists.append(full_filepath)

    # 결과 목록 리턴
    return result_file_lists

   
if __name__ == "__main__":
    p = argparse.ArgumentParser()
    p.add_argument("-f", type=str, help="<대상폴더>")
    p.add_argument("-e", type=str, help="<변경될 확장자>")
    args = p.parse_args()
    
    if args.f is None or args.e is None:
        print("사용법: python change.py -f <대상폴더> -e <변환될확장자>")
    else:
        # 실행인자 -f 로 넘어온 폴더 하위의 모든 이미지 파일을
        # 실행인자 -e 로 넘어온 포맷으로 변경합니다.
        # 실제 포맷이 변경된 이미지는 이미지가 있는 폴더에 convert 라는 폴더를 생성해서 저장됩니다.
        # 예) c:\temp\abcd\a.jpg => c:\temp\abcd\convert\a.png
        # 본인의 환경에 맞는 폴더로 변경하세요.
        target_dir = args.f
        new_format = args.e

        file_list = search_dir(target_dir)
        for file in file_list:
            new_folder = os.path.split(file)[0] + "\\convert"
            if not os.path.exists(new_folder):
                os.mkdir(new_folder)
            src_filename = os.path.splitext(file)[0]
            new_filename = new_folder + "\\" + src_filename.split("\\")[-1] + new_format
            img = Image.open(file)
            img.save(new_filename)
            

### 코드의 문제점
* 특정 대상 폴더에 이미지 포맷이 아닌 파일과 뒤섞여 있을때 오류를 발생합니다.
* 따라서 Image.open()을 할 때 해당 파일이 이미지 포맷이 아닌 경우에 대한 예외처리를 해야 합니다.
* pillow 에는 이미지의 유효성을 판단 할 수 있는 verify() 함수가 이미 만들어져 있습니다.
* verify 함수를 사용했을경우 이미지를 다시 open 해야 한다고 합니다.
* verify 함수 문서참조 : https://pillow.readthedocs.io/en/latest/reference/Image.html#PIL.Image.Image.verify

In [1]:
from PIL import Image
import os
import argparse


def search_dir(dirname):
    '''대상 폴더 내의 파일과 서브 폴더를 탐색하는 재귀 함수

    Args:
        dirname (str): 탐색 대상 폴더 경로

    Returns:
        list : 탐색한 결과 파일 리스트를 리턴
    '''
    # 결과를 리턴할 리스트 변수
    result_file_lists = []

    # dirname 의 경로의 파일과 폴더 목록을 구함
    filenames = os.listdir(dirname)

    # 파일과 폴더 목록 반복문
    for filename in filenames:
        # filename 에는 os.listdir(dirname) 에서 dirname 이후의 경로값과 파일명만 존재하기 때문에
        # 전체 풀 경로를 얻기 위해선 dirname 과 filename 을 합쳐야 함
        full_filepath = os.path.join(dirname, filename)

        # full_filepath 가 디렉토리라면..
        if os.path.isdir(full_filepath):
            # search_dir 인 자기 자신 함수를 다시 호출한다.(재귀함수)
            result_file_lists.extend(search_dir(full_filepath))
        # 파일인 경우 result_file_lists 에 추가
        else:
            result_file_lists.append(full_filepath)

    # 결과 목록 리턴
    return result_file_lists

   
if __name__ == "__main__":
    p = argparse.ArgumentParser()
    p.add_argument("-f", type=str, help="<대상폴더>")
    p.add_argument("-e", type=str, help="<변경될 확장자>")
    args = p.parse_args()
    
    if args.f is None or args.e is None:
        print("사용법: python change.py -f <대상폴더> -e <변환될확장자>")
    else:
        # 실행인자 -f 로 넘어온 폴더 하위의 모든 이미지 파일을
        # 실행인자 -e 로 넘어온 포맷으로 변경합니다.
        # 실제 포맷이 변경된 이미지는 이미지가 있는 폴더에 convert 라는 폴더를 생성해서 저장됩니다.
        # 예) c:\temp\abcd\a.jpg => c:\temp\abcd\convert\a.png
        # 본인의 환경에 맞는 폴더로 변경하세요.
        target_dir = args.f
        new_format = args.e

        # 대상 폴더의 하위폴더와 파일 리스트를 재귀함수를 통해 모두 구해옵니다.
        file_list = search_dir(target_dir)
        
        # 폴더/파일 리스트 요소 반복
        for file in file_list:
            # 해당 이미지 파일 하위에 convert 폴더에 저장을 합니다.
            new_folder = os.path.split(file)[0] + "\\convert"
            
            # convert 폴더가 존재 하지 않으면 새로 생성합니다.
            if not os.path.exists(new_folder):
                os.mkdir(new_folder)

            # c:\temp\test.jpg => c:\temp\convert\test.png 
            # 처럼 변경하기 위해 문자열을 분해조립 합니다.
            # splitext()[0] 하면 c:\temp\test 까지 나옵니다.
            src_filename = os.path.splitext(file)[0]
            
            # src_filename.split("\\") 하면
            # c:\temp\test 를 ["c:\temp", "test"] 로 분리하기 때문에
            # 여기서 -1 은 맨 마지막 요소가 되니 test 를 가져옵니다.
            new_filename = new_folder + "\\" + src_filename.split("\\")[-1] + new_format

            # 이미지 처리시 오류에 대한 예외처리를 해야 합니다.
            try:
                # 이미지를 open()
                img = Image.open(file)
                # 이미지 유효성을 체크 합니다.
                # 실제 txt, doc 등 이미지와 관련 없는 포맷은 open 시에 오류가 발생하겠지만
                # 정상적인 이미지 포맷이지만 읽을 수 없는 깨진파일 같은것은 open 시에 오류가 발생하지 않습니다.
                # 그러나 pillow 의 verify() 함수는 이렇게 정상적인 포맷이나 깨진 이미지까지 유효성을 체크합니다.
                # verify() 함수가 오류를 발생시킵니다.
                img.verify()
                img.close()

                # verify() 함수를 사용하고 이미지를 처리 하려면 다시 reopen()을 해야합니다.
                img = Image.open(file)

                # 해당 이미지를 새로운 경로에 새로운 확장자로 저장합니다.
                img.save(new_filename)
            except Exception as e:
                pass
            

사용법: python change.py -f <대상폴더> -e <변환될확장자>
